In [1]:
import scipy.stats as stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from math import sqrt
from sklearn import metrics
import json

In [2]:
data = pd.read_csv("./data/other_train.csv",index_col=0)

dict_array = []
string = '{"mean_glucose": 9999999,"std_glucose": 9999999,"kurtosis_glucose":9999999,"skewness_glucose":9999999,"mean_oxygen":9999999,"std_oxygen":9999999,"kurtosis_oxygen":9999999,"skewness_oxygen":9999999}'
data.medical_info.apply(lambda x: dict_array.append(json.loads(x.replace("'","\"").replace(":\"",":").replace("\",",",").replace("\"}","}") if isinstance(x,str) else string)))
medical_datas = pd.DataFrame(dict_array)

for item in medical_datas:
    data[item] = medical_datas[item].astype(float)




In [3]:
person = pd.read_csv("./data/personal_train.csv", index_col=0)

numeric = pd.DataFrame()
categoric = pd.DataFrame()

for column in data:
    if data[column].dtype == np.float64 or data[column].dtype == np.int64:
        numeric[column] = data[column]
    else:
        categoric[column] = data[column]
        
print(numeric.dtypes)
print('\n')
print(categoric.dtypes)


education-num       float64
capital-gain        float64
fnlwgt              float64
class               float64
hours-per-week      float64
capital-loss        float64
kurtosis_glucose    float64
kurtosis_oxygen     float64
mean_glucose        float64
mean_oxygen         float64
skewness_glucose    float64
skewness_oxygen     float64
std_glucose         float64
std_oxygen          float64
dtype: object


name              object
address           object
race              object
marital-status    object
occupation        object
pregnant          object
relationship      object
education         object
income            object
medical_info      object
native-country    object
workclass         object
dtype: object


In [4]:
# Stary parser
# numeric = numeric.replace([' ?','??',' ','?',' ??',None],np.NaN,regex=True)
# categoric = categoric.replace({'^[? ]*$' : None},regex=True)

numeric = numeric.replace({ 9999999 : np.nan },regex=False)
categoric = categoric.replace({'^[? ]*$' : np.nan},regex=True)

categoric.info()
numeric.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3982
Data columns (total 12 columns):
name              3983 non-null object
address           3983 non-null object
race              3970 non-null object
marital-status    3965 non-null object
occupation        3731 non-null object
pregnant          3961 non-null object
relationship      3967 non-null object
education         3967 non-null object
income            3965 non-null object
medical_info      3964 non-null object
native-country    3905 non-null object
workclass         3366 non-null object
dtypes: object(12)
memory usage: 404.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 3982
Data columns (total 14 columns):
education-num       3962 non-null float64
capital-gain        3970 non-null float64
fnlwgt              3966 non-null float64
class               3966 non-null float64
hours-per-week      3968 non-null float64
capital-loss        3969 non-null float64
kurtosis_glucose    396

In [5]:
numeric['name'] = data['name']
print(numeric.dtypes)
print(numeric)
print('\n')
print(categoric.dtypes)
print(categoric)

education-num       float64
capital-gain        float64
fnlwgt              float64
class               float64
hours-per-week      float64
capital-loss        float64
kurtosis_glucose    float64
kurtosis_oxygen     float64
mean_glucose        float64
mean_oxygen         float64
skewness_glucose    float64
skewness_oxygen     float64
std_glucose         float64
std_oxygen          float64
name                 object
dtype: object
      education-num  capital-gain    fnlwgt  class  hours-per-week  \
0               9.0           0.0  183279.0    0.0            40.0   
1              10.0           0.0  130724.0    1.0            25.0   
2              10.0           0.0  180284.0    0.0            50.0   
3              13.0           0.0  126060.0    0.0            40.0   
4               7.0           0.0  274657.0    1.0            40.0   
5              14.0           0.0  156743.0    0.0            76.0   
6              11.0           0.0  249463.0    1.0            20.0   
7     

In [6]:
numeric = numeric.groupby('name').aggregate('mean')

categoric = categoric.groupby('name').aggregate('first')

other = categoric.merge(numeric, on=['name'])
other

,address,race,marital-status,occupation,pregnant,relationship,education,income,medical_info,native-country,...,hours-per-week,capital-loss,kurtosis_glucose,kurtosis_oxygen,mean_glucose,mean_oxygen,skewness_glucose,skewness_oxygen,std_glucose,std_oxygen
name,,,,,,,,,,,,,,,,,,,,,
Aaron Davis,"379 Rodriguez Inlet\r\nFisherbury, MD 83114",Asian-Pac-Islander,Married-civ-spouse,Craft-repair,f,Husband,Some-college,>50K,"{'mean_glucose':'111.5546875','std_glucose':'5...",United-States,...,40.0,0.0,0.390191,21.400696,111.554688,0.530936,-0.176512,479.743759,52.480772,10.232655
Aaron Dini,"105 Medina Springs Suite 462\r\nEmilyland, KS ...",White,Divorced,Machine-op-inspct,F,Unmarried,HS-grad,<=50K,"{'mean_glucose':'110.2578125','std_glucose':'5...",United-States,...,40.0,0.0,0.280454,10.702973,110.257812,2.142140,-0.260235,149.848180,56.334554,13.440810
Aaron Gann,59685 Thomas Extension Apt. 142\r\nFieldsborou...,White,Married-civ-spouse,Sales,FALSE,Husband,Bachelors,<=50K,"{'mean_glucose':'128.1875','std_glucose':'49.8...",United-States,...,40.0,0.0,-0.016114,12.270117,128.187500,1.683946,-0.322560,198.354692,49.815088,12.169941
Aaron Gregg,"10883 Williams Locks Suite 821\r\nEast Carmen,...",White,Divorced,Craft-repair,f,Not-in-family,HS-grad,<=50K,"{'mean_glucose':'101.3828125','std_glucose':'5...",United-States,...,40.0,0.0,0.361766,8.375144,101.382812,3.933110,-0.168472,80.188004,51.082672,21.785797
Aaron Lindley,"564 Stevenson Park Apt. 839\r\nRodriguezville,...",White,Never-married,Sales,f,Own-child,Some-college,<=50K,"{'mean_glucose':'75.640625','std_glucose':'34....",United-States,...,35.0,0.0,1.073438,5.188312,75.640625,6.118729,4.466385,28.565339,34.337192,28.610537
Aaron Manning,"9503 Christy Meadow\r\nAntoniostad, DC 31369",White,Never-married,Prof-specialty,f,Not-in-family,Bachelors,<=50K,"{'mean_glucose':'126.5546875','std_glucose':'4...",United-States,...,40.0,0.0,0.105554,3.608318,126.554688,11.919732,0.016703,12.289702,48.738590,41.445156
Aaron Moody,"5443 Kurt Estate\r\nHillbury, MS 07587",White,Married-civ-spouse,Exec-managerial,f,Husband,Some-college,<=50K,"{'mean_glucose':'122.9609375','std_glucose':'5...",United-States,...,20.0,0.0,-0.100267,13.478446,122.960938,1.456522,-0.245821,248.420253,50.770840,11.160307
Aaron Noman,"PSC 0351, Box 8841\r\nAPO AA 86063",White,Married-civ-spouse,Sales,f,Husband,Some-college,>50K,"{'mean_glucose':'125.6875','std_glucose':'54.4...",United-States,...,40.0,0.0,0.065459,6.475870,125.687500,3.986622,-0.538646,45.207830,54.400676,23.026627
Aaron Robinson,"16767 Medina Manor Suite 889\r\nCarlosshire, M...",White,Never-married,Transport-moving,T,Own-child,HS-grad,<=50K,"{'mean_glucose':'99.7578125','std_glucose':'41...",United-States,...,30.0,0.0,0.634865,8.910677,99.757812,2.622910,0.992106,89.312679,41.409869,18.614801


In [7]:
data_merged = other.merge(person, on=['name', 'address'])
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3933 entries, 0 to 3932
Data columns (total 29 columns):
name                3933 non-null object
address             3933 non-null object
race                3933 non-null object
marital-status      3933 non-null object
occupation          3703 non-null object
pregnant            3931 non-null object
relationship        3932 non-null object
education           3933 non-null object
income              3932 non-null object
medical_info        3932 non-null object
native-country      3872 non-null object
workclass           3341 non-null object
education-num       3933 non-null float64
capital-gain        3933 non-null float64
fnlwgt              3932 non-null float64
class               3933 non-null float64
hours-per-week      3932 non-null float64
capital-loss        3933 non-null float64
kurtosis_glucose    3932 non-null float64
kurtosis_oxygen     3932 non-null float64
mean_glucose        3932 non-null float64
mean_oxygen         393

In [8]:
# data_merged['pregnant'].value_counts()

In [9]:
data_merged = data_merged.replace(['FALSE','F'],'f',regex=True)
data_merged = data_merged.replace(['TRUE','T'],'t',regex=True)
# data_merged['pregnant'].value_counts()

In [10]:
# data_merged['income'].value_counts()

In [11]:
data_merged = data_merged.replace([' <=50K'],'<=50K',regex=True)
data_merged = data_merged.replace([' >50K'],'>50K',regex=True)
# data_merged['income'].value_counts()

In [12]:
# data_merged['relationship'].value_counts()

In [13]:
data_merged = data_merged.replace([' Other-relative'],' Other_relative',regex=True)
data_merged = data_merged.replace([' Not-in-family'],' Not_in_family',regex=True)
data_merged = data_merged.replace([' Own-child'],' Own_child',regex=True)
# data_merged['relationship'].value_counts()

In [14]:
# data_merged['date_of_birth'].value_counts()

In [15]:
for i, row in data_merged.iterrows():
        
    date = parser.parse(data_merged.iloc[i]['date_of_birth'].split(' ')[0])
    day = date.day
    month = date.month
    year = date.year
    
    if (year > 2019):
        year = year - 100

    data_merged.at[i,'date_of_birth'] = str(day) + "/" + str(month) + "/" + str(year)
            
    if ((pd.isna(row['age'])) | (row['age'] > 115) | (row['age'] < 0)):
        data_merged.at[i,'age'] = 2019 - year
        
# data_merged

In [16]:
numeric = data_merged.select_dtypes(include=['float64'])

# Remove NULL value for numeric columns except mean_glucose
for item in data_merged:
    if data_merged[item].dtype == np.float64 and item != 'mean_glucose':
        imp = Imputer(missing_values=np.NaN, strategy='mean')
        data_merged[item] = imp.fit_transform(data_merged[[item]])
        
# data_merged.info()

In [17]:
categoric =  data_merged.select_dtypes(include=['object'])

# Remove NULL value for categorical columns
for item in data_merged:
    if (data_merged[item].dtype == np.object):
        imp = Imputer(missing_values=np.nan, strategy='most_frequent')
        data_merged[item] = imp.fit_transform(data_merged[[item]])

In [18]:
objectsTrain = numeric[numeric.isna().any(axis=1) == False]
objectsPredict = numeric[numeric.isna().any(axis=1) == True]


In [23]:
objectsPredict

,education-num,capital-gain,fnlwgt,class,hours-per-week,capital-loss,kurtosis_glucose,kurtosis_oxygen,mean_glucose,mean_oxygen,skewness_glucose,skewness_oxygen,std_glucose,std_oxygen,age
766,9.0,0.0,167919.0,0.0,NaN,0.0,0.503002,6.716207,108.273438,3.723244,2.078418,54.194037,34.664624,19.496749,53.0
1529,9.0,3818.0,144056.0,0.0,40.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.0
2124,9.0,0.0,NaN,0.0,40.0,0.0,0.667347,21.988079,87.429688,0.516722,3.598325,553.231999,33.739277,9.061692,34.0


In [19]:
# KNN regression to know how many groups make


# Xtrain,Xtest,Ytrain,Ytest = split(objectsTrain['skewness_glucose'],objectsTrain['mean_glucose'],test_size=0.3)
# 
# scores = []
# Xtrain = Xtrain.values.reshape(-1,1)
# #Ytrain = Ytrain.values.reshape(-1,1)
# Xtest = Xtest.values.reshape(-1,1)
# #Ytest = Ytest.values.reshape(-1,1)
# for k in range(1,300):
#     model = neigh(n_neighbors = k)    
#     model.fit(Xtrain,Ytrain)
#     Ypred = model.predict(Xtest)
#     scores.append(sqrt(metrics.mean_squared_error(Ytest,Ypred)))

In [20]:
# print(scores.index(min(scores)),' : ', min(scores))
# curve = pd.DataFrame(scores)
# curve.plot()

In [21]:
# Real prediction on mean_glucose with skewness_glucose and use k group
# !!!IMPORTANT!!! need to add into datas

Xpred = objectsPredict['skewness_glucose'].values.reshape(-1,1)
Ypred = objectsPredict['mean_glucose']

Xtrain = objectsTrain['skewness_glucose'].values.reshape(-1,1)
Ytrain = objectsTrain['mean_glucose'].values.reshape(-1,1)
k = 98
model = neigh(n_neighbors = k)    
model.fit(Xtrain,Ytrain)

Ypred = model.predict(Xpred)


Ypred = pd.DataFrame(data = Ypred, columns = ['mean_glucose'], index = Xpred.index.copy())
objectsPredict_out = pd.merge(objectsPredict, Ypred, how = 'left', left_index = True, right_index = True)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# def identify_outliers(a):
#     lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
#     upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
#     
#     return a[(a > upper) | (a < lower)]
# 
# # Get only float64 columns from datas
# numeric = data_merged.select_dtypes(include=['float64'])

In [ ]:
# Print all distplot from numeric datas

# for i in numeric:
#     plt.figure(i)
#     sns.distplot(numeric[i])

In [ ]:
## Replace all numeric outliers
## !!!IMPORTANT!!! need add another methods to replace outliers like transform value by using sqrt or logaritmic function

# for i in numeric:
#     median = numeric[i].median()
#     std = numeric[i].std()  
#     outliers = identify_outliers(numeric[i])
#     numeric[i] = numeric[i].replace(outliers.index,np.nan)
#     numeric[i].fillna(median,inplace=True)

In [ ]:
##
## !!!IMPORTANT!!! need some persistance on models(replace NULL value, replace Outliers)
